In [33]:
import numpy as np
from pyswmm import Simulation, Nodes, Links
import pandas as pd
from swmm_api import read_inp_file, read_out_file
from swmm_api.output_file.extract import SwmmOutExtractWarning
from routing import Router, approximate_value
from tqdm.auto import tqdm
import random
import warnings
from wipes import get_snagging_function, process_pile

warnings.filterwarnings("ignore", category=SwmmOutExtractWarning)

## Input Data
please insert model- and output-paths here

In [2]:
model_path = r"C:\Users\albert\PycharmProjects\PacketSWMM\sample_data\sample_model.inp"
out_path = r"C:\Users\albert\PycharmProjects\PacketSWMM\sample_data\sample_model.out"

# Snagging function
define the function that determines with what probability a packet snaggs

In [25]:
probability_by_velocity = lambda v: 0.5

# Blocking function
Define the blockage / opening of conduit depending on pile size

In [53]:
blockage_from_pile = lambda pile: pile / (20+pile)

# Runner
This cell runs the model, interrupts in defined intervals, reads output data and adapts the model parameters

In [9]:
target_node ="MH3293004253"
pile_decay_rate = 0.1  # 1/hr
update_freq = "10H"
update_time = pd.to_timedelta(update_freq)

router = Router()
router.get_network_from_inpfile(model_path)
succ_node = router.g_network.successors(target_node)
target_link = router.g_network.edges[succ_node, succ_node]["name"]
rtable = router.route_table(target=target_node)
pile = 0

with Simulation(model_path) as sim:
    updates = pd.date_range(sim.start_time, sim.end_time, freq=update_freq)
    previous_time = sim.start_time
    super_previous_time = sim.start_time
    link_blockage = Links(sim)[target_link]
    
    for step in tqdm(sim):
        if sim.current_time in updates:
            # get new flows
            router.get_flows_from_outfile(out_path, super_previous_time, sim.current_time)
            # create snagging function
            snag = get_snagging_function(probability_by_velocity, flows=router.df_flows[target_node]) 
            
            # prepare and route routing table
            rtable = router.from_old_routingtable(rtable)
            df_packages = router.generate_packets(previous_time, sim.current_time)
            rtable = pd.concat([rtable, df_packages], axis=0)
            rtable = router.route_table(rtable)
            # extract arrivals at target node
            new_arrivals = rtable[target_node].dropna().sort_values()
            # process new arrivals at target node
            pile = process_pile(new_arrivals, pile, previous_time, sim.current_time, pile_decay_rate, snag)
            link_blockage.target_setting = blockage_from_pile(pile)
            
            # prepare next update
            super_previous_time = previous_time
            previous_time = sim.current_time
            pass
        pass

0it [00:00, ?it/s]